In [1]:
import chemistry_vae_symmetric_rnn
import data_loader
import selfies
import torch
import importlib
import yaml
import pandas as pd
import numpy as np
import torch.distributions as dist
import matplotlib.pyplot as plt
import networkx as nx
from pysmiles import read_smiles
import Levenshtein
import random


In [4]:
# here we want to read in data and tranform them into one_hot
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

folder_path = "../../Clean_VAE_file/datasets/"
file_name = "CSD_EES_DB_energies.csv"

full_path = folder_path + file_name

selfies_list, selfies_alphabet, largest_selfies_len, smiles_list, smiles_alphabet, largest_smiles_len = chemistry_vae_symmetric_rnn.get_selfie_and_smiles_encodings_for_dataset(full_path)

{'H': 1, 'F': 1, 'Cl': 1, 'Br': 1, 'I': 1, 'B': 3, 'B+1': 2, 'B-1': 4, 'O': 2, 'O+1': 3, 'O-1': 1, 'N': 3, 'N+1': 4, 'N-1': 2, 'C': 4, 'C+1': 5, 'C-1': 3, 'P': 5, 'P+1': 6, 'P-1': 4, 'S': 6, 'S+1': 7, 'S-1': 5, '?': 8}
                                                  smiles  energy_gap
0      COC1=C(C(OC1=O)c1ccccc1Cl)C(C)=NN=C(C)C1=C(OC)...     0.20713
1      CN1N(c2ccccc2)C(=O)C(=C1C)N=Cc1ccccc1C=NC1=C(C...     0.21467
2                   COc1cc(OC)c(C=CC(=O)c2ccccn2)c(OC)c1     0.21389
3            O=P1(c2ccccc2)c2ccc3ccccc3c2c2c3ccccc3ccc12     0.20755
4              COc1cc(Br)cc(C=NNC(=O)CC2=CNc3ccccc23)c1O     0.22060
...                                                  ...         ...
34405  C[NH+]1CCC23C4CCC(OS(=O)(=O)[O-])C2Oc2c(OC(C)=...     0.17994
34406                      [NH3+]C(C1CC1C(=O)[O-])C(O)=O     0.15398
34407                      [NH3+]C(C1CC1C(=O)[O-])C(O)=O     0.18568
34408  CSC1=NC2=C(NC(C(C(CC(=O)c3ccccc3)c3ccccc3)C(=N...     0.11361
34409  CC(CCC(=C)C(C)C

In [5]:
# define source file location
file_to_load =  "../../rnn_sym_crystals/stack_size3neurons_num2150/"
#file_to_load =  "./model runs/saved_models_run1/"
#file_to_load =  "./saved_models_run1/"
# training file name encoder
training_file_nameE = "500/E"
# training file name decoder
training_file_nameD = "500/D"
# load data
#load_data_trained = file_to_load + training_file_nameE
# Alphabet has 18 letters, largest molecule is 21 letters. (build this as an output function later ... )
largest_selfies_len_dataset = largest_selfies_len
largest_smiles_len_dataset = largest_smiles_len

#in_dimension = len(selfies_alphabet)*largest_selfies_len
in_dimension = len(smiles_alphabet)*largest_smiles_len



# load the trained encoder
vae_encoder = torch.load(file_to_load + training_file_nameE, map_location=torch.device(device="cpu"))
#print(vae_encoder)

# load the trained decoder
vae_decoder = torch.load(file_to_load + training_file_nameD, map_location=torch.device(device="cpu"))

In [6]:
selfies_alphabet = [
    "[nop]",
    "[Cl]",
    "[=As]",
    "[=Branch2]",
    "[NH2+1]",
    "[#Branch1]",
    "[C]",
    "[Branch1]",
    "[=Se]",
    "[S+1]",
    "[=N-1]",
    "[=C]",
    "[S]",
    "[C-1]",
    "[=N]",
    "[#Branch2]",
    "[#N]",
    "[=Branch1]",
    "[N+1]",
    "[I]",
    "[NH1+1]",
    "[Br]",
    "[=NH1+1]",
    "[O-1]",
    "[=P]",
    "[P-1]",
    "[=O+1]",
    "[SiH1]",
    "[B-1]",
    "[=O]",
    "[#C]",
    "[As]",
    "[Se-1]",
    "[Ring2]",
    "[F]",
    "[=N+1]",
    "[=Ring2]",
    "[=B-1]",
    "[N-1]",
    "[=S]",
    "[NH3+1]",
    "[Ring1]",
    "[Branch2]",
    "[=Ring1]",
    "[P+1]",
    "[=NH2+1]",
    "[S-1]",
    "[N]",
    "[O]",
    "[P]"
]

In [14]:
# useful functions

def translate_selfie(sequence):  
        SELFIESGenerated = ''

        for i in range(len(sequence)):
                SELFIESGenerated = SELFIESGenerated + selfies_alphabet[sequence[i]]
        return SELFIESGenerated

def translate_smile(sequence):  

        SELFIESGenerated = ""

        for i in range(len(sequence)):
                SELFIESGenerated = SELFIESGenerated + smiles_alphabet[sequence[i]]
        return SELFIESGenerated
        
def create_onehot_instance(selfie_input,largest_selfies_len,selfies_alphabet_in):

    inttest_hot, arraytest_hot = data_loader.selfies_to_hot(selfie_input,largest_selfies_len, selfies_alphabet_in)
    x = torch.from_numpy(arraytest_hot).flatten().float().unsqueeze(0)
    return x



def create_latent_space_vector(selfie_input,largest_selfies_len,selfies_alphabet_in):

    x = create_onehot_instance(selfie_input,largest_selfies_len,selfies_alphabet_in)
    x = x.unsqueeze(0)

    z =set()
    vae_encoder.eval()
    vae_decoder.eval()
    z, mu, log_var = vae_encoder(x)

    return mu.unsqueeze(0)





def decode_from_latentspace(latent_point_in, largest_selfies_len_in, selfies_alphabet_len, method):


        vae_decoder.eval()
        vae_encoder.eval()

        sequence = []

        hidden = vae_decoder.init_hidden(batch_size=1)

         
        for seq_index in range(largest_selfies_len_in):
                out_one_hot_line, hidden = vae_decoder(latent_point_in, hidden)

                if method == 0:
                        sequence.append(out_one_hot_line.argmax())

                elif method ==1:
                        # Apply softmax and sample from the distribution to get the next token
                        softmax = torch.nn.Softmax(dim=2)
                        probabilities = softmax(out_one_hot_line)
                        categorical_dist = dist.Categorical(probabilities)
                        sample = categorical_dist.sample()
                        sequence.append(sample)

                else:
                        print("method is 0 for argmax or 1 for stat sampling")
        
        

        return sequence

In [15]:
# pick a molecule

selfie_input = selfies_list[200]
print('SELFIES input:', selfies.decoder(selfie_input))

SELFIES input: NC1=CC=CC=C1C#CC2=CC=CC=C2C#CC3=CC=C(C=C3)C#CC4=CC=CC=C4C#CC5=CC=CC=C5N


In [16]:
print("encoded in", selfies.decoder(selfie_input))

for i in range(10):
    coded_selfie_from_latent_space = decode_from_latentspace(create_latent_space_vector(selfie_input,largest_selfies_len,selfies_alphabet) , largest_selfies_len, len(selfies_alphabet),0) 
    selfie_from_latent_space = translate_selfie(coded_selfie_from_latent_space)
    print("reconstructed out", selfies.decoder(selfie_from_latent_space))

encoded in NC1=CC=CC=C1C#CC2=CC=CC=C2C#CC3=CC=C(C=C3)C#CC4=CC=CC=C4C#CC5=CC=CC=C5N
reconstructed out NC1=CC=CC=C1C#CC2=CC=CC=C2C#CC3=CC=C(C=C3)C#CC4=CC=CC=C4C#CC5=CC=CC=C5N
reconstructed out NC1=CC=CC=C1C#CC2=CC=CC=C2C#CC3=CC=C(C=C3)C#CC4=CC=CC=C4C#CC5=CC=CC=C5N


KeyboardInterrupt: 

In [17]:
##qm9 levenshtein:

smiles_input = []
recon_smiles = []




for i in range(100):
    ran_int = random.randint(0, len(selfies_list))

    inp_self = selfies_list[ran_int]
    smiles_input.append(selfies.decoder(inp_self))

    
    latent_vector = create_latent_space_vector(inp_self,largest_selfies_len,selfies_alphabet)
    coded_selfie_from_latent_space = decode_from_latentspace(latent_vector, largest_selfies_len, len(selfies_alphabet),0) 
    selfie_from_latent_space = translate_selfie(coded_selfie_from_latent_space)
    recon_smiles.append(selfies.decoder(selfie_from_latent_space))

    if i % 10 == 0:
        print('step:', i)


step: 0
step: 10
step: 20
step: 30
step: 40
step: 50
step: 60
step: 70
step: 80
step: 90


In [18]:
levenshteins = []

for i in range(len(smiles_input)):
    original_smiles = smiles_input[i]
    output_smiles = recon_smiles[i]

    # Calculate the Levenshtein distance
    levenshtein_distance = Levenshtein.distance(original_smiles, output_smiles)
    levenshteins.append(levenshtein_distance)


print('mean Levenshtein distance:', np.mean(levenshteins), '+/-', np.std(levenshteins))

mean Levenshtein distance: 17.2 +/- 18.46889276594566
